# Tutorial 1: Introduction to Gridap.jl - From Weak Form to Elemental Matrix and Vector

Welcome to the first tutorial in the CMOE series on computational methods for multiphysics problems! In this tutorial, you will learn the fundamentals of using **Gridap.jl**, a powerful finite element library in Julia, to solve partial differential equations (PDEs).

> **📘 About this tutorial**: The intention of this tutorial is to go in depth into the integration/assembly process, from the weak formulation down to the elemental matrix and vector computations. The set of tutorials provided in this repository are complementary to the [official Gridap Tutorials](https://gridap.github.io/Tutorials/dev/). We recommend the reader to have a look at those first.

> **⚠️ Note**: The text in this tutorial has been generated by LLMs. If you find any inconsistencies, please report them through an issue in the repository.

## Table of Contents

**Part 1: High-Level Implementation**
- 1.1 [Setting Up the Julia Environment](#11-setting-up-the-julia-environment)
- 1.2 [Creating a Computational Domain with Cartesian Meshes](#12-creating-a-computational-domain-with-cartesian-meshes)
- 1.3 [Defining Reference Finite Elements and Finite Element Spaces](#13-defining-reference-finite-elements-and-finite-element-spaces)
- 1.4 [Defining Triangulation and Integration Measures](#14-defining-triangulation-and-integration-measures)
- 1.5 [Weak Form Definition and Solution](#15-weak-form-definition-and-solution)
- 1.6 [Inspecting the Assembled System Matrices](#16-inspecting-the-assembled-system-matrices)

**Part 2: Low-Level Implementation**

- 2.1 [Understanding Basis Functions: Reference vs Physical Domain](#21-understanding-basis-functions-reference-vs-physical-domain)
- 2.2 [Finite Element Functions: Free vs Fixed Degrees of Freedom](#22-finite-element-functions-free-vs-fixed-degrees-of-freedom)
- 2.3 [From Weak Forms to Elemental Matrices and Vectors](#23-from-weak-forms-to-elemental-matrices-and-vectors)

## Learning Objectives

By the end of this tutorial, you will be able to:
- Create computational domains using Cartesian meshes
- Define finite element spaces and reference elements
- Formulate weak forms of PDEs
- Assemble and solve linear systems
- Understand the structure of elemental matrices and vectors
- Inspect element-level contributions before global assembly
- Visualize solutions using VTK output

We will go deep into what happens under the hood within the assembly process in Gridap.jl, examining how weak forms are transformed into elemental matrices and vectors. In further tutorials we will go deeper on how the numerical integration and assembly process is performed under the hood.

## Problem Statement

We will solve the classic **Poisson equation** on a 2D domain:

$$-\Delta u = f \quad \text{in } \Omega$$

with appropriate boundary conditions. This fundamental problem appears in many engineering applications including heat transfer, electrostatics, and potential flow.

## Prerequisites

- Basic understanding of finite element methods
- Familiarity with Julia programming language
- Knowledge of partial differential equations
- Understanding of weak formulations

Let's get started!

---

# Part 1: High-Level Implementation

In this first part, we'll use Gridap's high-level interface to solve a Poisson equation. This demonstrates the elegant, mathematical syntax that makes Gridap powerful for rapid prototyping and development.

---

## 1. Setting Up the Julia Environment

Before we start working with Gridap.jl, we need to properly set up our Julia environment. This involves two critical steps:

### 1.1 Activating the Project Environment 
```julia
Pkg.activate("..")
```

When you activate a project environment, Julia switches from the global package registry to a **local project-specific environment**. This means:

- **Reproducibility**: The project uses the exact package versions specified in `Project.toml` and `Manifest.toml` files
- **Isolation**: Changes to packages in this project won't affect other Julia projects on your system
- **Consistency**: Everyone working on this tutorial will use the same package versions, avoiding "works on my machine" issues

The `".."` argument tells Julia to activate the environment in the parent directory.

### 1.2 Instantiating the Environment
```julia
Pkg.instantiate()
```

The `instantiate()` command ensures all required packages are installed with their correct versions:

- **Downloads missing packages**: If any packages specified in `Project.toml` are not installed, they will be downloaded
- **Resolves dependencies**: Ensures all package dependencies are compatible and available
- **Builds packages**: Compiles any binary dependencies that packages might need

Think of this as similar to `npm install` (Node.js), `pip install -r requirements.txt` (Python), or `maven install` (Java).

**Note**: You only need to run `instantiate()` once when you first clone or download this tutorial. After that, simply activating the environment is sufficient, unless the package dependencies change.

In [28]:
using Pkg
Pkg.activate("..")
Pkg.instantiate()

  Activating project at `~/Progs/git_repos/CMOE/CMOE-Tutorials`
project at `~/Progs/git_repos/CMOE/CMOE-Tutorials`


## 2. Creating a Computational Domain with Cartesian Meshes

In finite element analysis, the first step is to discretize our computational domain into a mesh. Gridap.jl provides the `CartesianDiscreteModel` function to create structured Cartesian meshes efficiently.

### 2.1 Understanding the Domain Setup

Let's break down the components:

1. **Domain Definition**: `domain = (0, L, 0, L)`
   - Defines a square domain from $(0,0)$ to $(L,L)$ in 2D space
   - The tuple format is `(x_min, x_max, y_min, y_max)`
   - For our example, with `L = 2.0`, we create a $2.0 \times 2.0$ square

2. **Mesh Partition**: `partition = (n, n)`
   - Specifies how many cells to create in each direction
   - With `n = 2`, we divide the domain into $2 \times 2 = 4$ square cells
   - Each cell will have dimensions $\frac{L}{n} \times \frac{L}{n} = 1.0 \times 1.0$

3. **Creating the Model**: `model = CartesianDiscreteModel(domain, partition)`
   - Generates the discrete model containing:
     - **Nodes (vertices)**: $(n+1) \times (n+1) = 9$ nodes in our case
     - **Cells (elements)**: $n \times n = 4$ quadrilateral elements
     - **Boundary tags**: Automatically labels domain boundaries for applying boundary conditions

### 2.2 Visualization with VTK

The `writevtk(model, "model")` function exports the mesh to VTK format:
- Creates files `model_0.vtu`, `model_1.vtu`, `model_2.vtu` (for different topological dimensions)
- These files can be visualized using [ParaView](https://www.paraview.org/) or other VTK-compatible software
- Useful for inspecting the mesh quality and structure before solving

> **📘 Note**: Cartesian meshes are ideal for rectangular domains and provide uniform element sizes, which is excellent for learning and debugging. For more complex geometries, Gridap also supports unstructured meshes imported from external mesh generators (like GMSH).

In [29]:
using Gridap
L = 2.0
n = 2
domain = (0,L,0,L)
partition = (n,n)
model = CartesianDiscreteModel(domain,partition)
writevtk(model,"model")

3-element Vector{Vector{String}}:
 ["model_0.vtu"]
 ["model_1.vtu"]
 ["model_2.vtu"]

## 3. Defining Reference Finite Elements and Finite Element Spaces

After creating the mesh, we need to define the **finite element spaces** where our solution will live. This involves two key concepts: the reference finite element and the function spaces built from it.

### 3.1 Reference Finite Element

```julia
reffe = ReferenceFE(lagrangian, Float64, 1)
```

The `ReferenceFE` defines the type of finite element basis functions we'll use:

- **`lagrangian`**: Specifies Lagrangian (nodal) interpolation
  - Basis functions have the property $\phi_i(x_j) = \delta_{ij}$ (Kronecker delta)
  - Values at nodes directly correspond to degrees of freedom
  
- **`Float64`**: The data type for storing values (double precision floating point)

- **`1`**: The polynomial order of the basis functions
  - Order 1 = linear elements (straight edges, planar faces)
  - For 2D quadrilaterals, this gives bilinear basis functions
  - Each element will have 4 nodes (one at each corner)

### 3.2 Test Function Space (V)

```julia
V = FESpace(model, reffe; conformity=:H1, dirichlet_tags="boundary")
```

The test space `V` is where we search for our solution. Key parameters:

- **`model`**: Links the space to our computational mesh
- **`reffe`**: Uses the reference element defined above
- **`conformity=:H1`**: Ensures functions are continuous across element boundaries
  - $H^1$ conformity means $u \in H^1(\Omega)$: functions with square-integrable derivatives
  - Essential for problems involving gradients (like our Poisson equation)
- **`dirichlet_tags="boundary"`**: Identifies where to apply Dirichlet boundary conditions
  - Tags all nodes on the domain boundary
  - These will be treated as constrained degrees of freedom

### 3.3 Trial Function Space (U)

```julia
U = TrialFESpace(V, x -> x[1])
```

The trial space `U` is constructed from the test space `V` with prescribed boundary values:

- **`V`**: Inherits structure from the test space
- **`x -> x[1]`**: Boundary condition function
  - For each point `x` on the boundary, sets $u = x_1$ (the x-coordinate)
  - This creates a non-homogeneous Dirichlet condition: $u(x_1, x_2) = x_1$ on $\partial\Omega$
  - Interior nodes remain free (unknowns to solve for)

### 3.4 Galerkin Method Structure

In the Galerkin finite element method:
- **Trial functions** ($u \in U$): Satisfy the boundary conditions and represent candidate solutions
- **Test functions** ($v \in V$): Satisfy homogeneous boundary conditions ($v = 0$ on $\partial\Omega$) and are used to form the weak formulation

The distinction ensures that when we test the residual with functions from $V$, the boundary conditions are properly enforced through the trial space $U$.

In [30]:
# Solve the Poisson problem
reffe = ReferenceFE(lagrangian,Float64,1)
V = FESpace(model,reffe;conformity=:H1,dirichlet_tags="boundary")
U = TrialFESpace(V,x->x[1])

TrialFESpace()

## 4. Defining Triangulation and Integration Measures

Now that we have defined our finite element spaces, we need to set up the infrastructure for performing numerical integration over the domain. This is essential for computing the integrals that appear in the weak formulation of our PDE.

### 4.1 Triangulation

```julia
Ω = Triangulation(model)
```

The `Triangulation` represents the computational domain decomposed into cells (elements):

- **`Ω`** (capital Omega): Denotes the computational domain
- Creates a triangulation object from our discrete model
- Provides the geometric information needed for integration:
  - Cell connectivity
  - Node coordinates
  - Jacobian transformations between reference and physical  (information required to do this operation)
  - ...
- In our case with `n=2`, we have 4 quadrilateral cells

### 4.2 Integration Measure

```julia
dΩ = Measure(Ω, 2)
```

The `Measure` defines how we perform numerical integration over the domain:

- **`dΩ`**: Represents the differential volume element (in mathematical notation: $d\Omega$)
- **`Ω`**: The triangulation over which we integrate
- **`2`**: The quadrature order (degree of precision)
  - Determines how many integration points per cell
  - Order 2 is exact for polynomials up to degree 2
  - For bilinear basis functions (order 1), this is sufficient for the stiffness matrix

### 4.3 Numerical Integration in Gridap

When we write integrals like $\int_\Omega \nabla u \cdot \nabla v \, d\Omega$ in Gridap, the `dΩ` measure handles:

1. **Quadrature rules**: Selects integration points and weights within each cell
2. **Information of the integration domain** in which different opperations are performed, e.g.:
  - **Coordinate transformation**: Maps from reference element to physical cells
  - **Jacobian computation**: Accounts for the volume change in the mapping
  - **Assembly**: Accumulates contributions from all cells

The beauty of Gridap's syntax is that `dΩ` makes the code look like the mathematical weak formulation!

> **📘 Note**: The quadrature order should be chosen based on the polynomial degree of the integrand. For linear elements with gradient operators, order 2 is typically sufficient. Higher-order elements would require higher quadrature orders.

In [31]:
Ω = Triangulation(model)
dΩ = Measure(Ω,2)

GenericMeasure()

## 5. Weak Form Definition and Solution

With all the components in place, we can now define the weak formulation of our Poisson equation and solve the resulting linear system.

### 5.1 Source Term

```julia
f(x) = 1.0
```

This defines the source term (right-hand side) of our Poisson equation $-\Delta u = f$:
- Constant source term $f = 1.0$ throughout the domain
- More complex functions could depend on position: `f(x) = sin(π*x[1])*cos(π*x[2])`

### 5.2 Bilinear Form (Left-Hand Side)

```julia
a(u,v) = ∫( ∇(v)⋅∇(u) )dΩ
```

The bilinear form `a(u,v)` represents the left-hand side of the weak formulation:

$$a(u,v) = \int_\Omega \nabla v \cdot \nabla u \, d\Omega$$

- **`∇(u)`** and **`∇(v)`**: Compute gradients of trial and test functions
- **`⋅`**: Dot product of the gradient vectors
- **`∫(...)dΩ`**: Integration over the domain using the measure we defined
- This corresponds to the weak form of $-\Delta u$ after integration by parts

### 5.3 Linear Form (Right-Hand Side)

```julia
l(v) = ∫( f*v )dΩ
```

The linear form `l(v)` represents the right-hand side:

$$l(v) = \int_\Omega f \, v \, d\Omega$$

- Multiplies the source term `f` by the test function `v`
- Integrated over the entire domain

### 5.4 Creating the FE Operator

```julia
op = AffineFEOperator(a, l, U, V)
```

The `AffineFEOperator` assembles the discrete system:
- Takes the bilinear form `a` and linear form `l`
- Uses trial space `U` and test space `V`
- Constructs the linear system: $\mathbf{A}\mathbf{u} = \mathbf{b}$
  - $\mathbf{A}_{ij} = a(\phi_j, \psi_i)$: Stiffness matrix
  - $\mathbf{b}_i = l(\psi_i)$: Load vector
- Applies boundary conditions from the trial space `U`

### 5.5 Solving the System

```julia
uh = solve(op)
```

The `solve` function:
- Solves the linear system $\mathbf{A}\mathbf{u} = \mathbf{b}$
- Returns a finite element function `uh` (the discrete solution)
- Uses appropriate linear solver (typically direct solver for small problems, iterative for large ones)

### 5.6 Visualization

```julia
writevtk(Ω, "solution", cellfields=["uh"=>uh])
```

Exports the solution to VTK format:
- Creates `solution.vtu` file
- Associates the solution field `uh` with the name "uh"
- Can be opened in ParaView to visualize the solution field

> **📘 Note**: Gridap's syntax closely mirrors mathematical notation, making it intuitive to translate weak formulations directly into code. The `∫` symbol can be typed with `\int<TAB>` in Julia.

In [32]:
f(x) = 1.0
a(u,v) = ∫( ∇(v)⋅∇(u) )dΩ
l(v) = ∫( f*v )dΩ
op = AffineFEOperator(a,l,U,V)
uh = solve(op)  
writevtk(Ω,"solution",cellfields=["uh"=>uh])

(["solution.vtu"],)

## 6. Inspecting the Assembled System Matrices

After solving the problem, it's instructive to examine the assembled linear system to understand what Gridap has constructed under the hood.

### 6.1 Extracting System Components

```julia
A = get_matrix(op)
b = get_vector(op)
```

These functions extract the discrete linear system from the FE operator:

- **`A`**: The global stiffness matrix (sparse format)
  - Size: $n_{dof} \times n_{dof}$ where $n_{dof}$ is the number of degrees of freedom
  - Symmetric positive definite for the Poisson problem
  - Most entries are zero (sparse structure) due to local support of basis functions
  
- **`b`**: The global load vector
  - Size: $n_{dof} \times 1$
  - Contains contributions from the source term and boundary conditions

### 6.2 Displaying the System

```julia
display(Matrix(A))
display(Vector(b))
```

Converting to dense formats for inspection:

- **`Matrix(A)`**: Converts sparse matrix to dense format for visualization
  - Shows the full matrix including zeros
  - Reveals the sparsity pattern and connectivity
  - For our $2 \times 2$ mesh, we have 9 nodes but some are constrained by boundary conditions
  
- **`Vector(b)`**: Displays the load vector as a standard array

### 6.3 What to Look For

When examining the assembled system:

1. **Matrix sparsity**: Non-zero entries only where basis functions overlap
2. **Symmetry**: For self-adjoint problems like Poisson, $A$ should be symmetric
3. **Diagonal dominance**: Often present in well-posed elliptic problems
4. **Boundary conditions**: Reflected in rows/columns corresponding to boundary nodes
5. **Load vector**: Should reflect the integrated source term over elements

> **📘 Note**: For large problems, converting to dense format is impractical. In production code, sparse matrices should be kept in sparse format. Here we convert for educational purposes only.

In [33]:
using SparseArrays
A = get_matrix(op)
b = get_vector(op)
display(Matrix(A))
display(Vector(b))


1×1 Matrix{Float64}:
 2.6666666666666656

1-element Vector{Float64}:
 3.666666666666665

### Understanding the System Size

The system is **1×1** because we only have **1 free degree of freedom**.

With a $2 \times 2$ mesh, we have $(n+1) \times (n+1) = 3 \times 3 = 9$ total nodes. However, boundary conditions are applied to all boundary nodes:

- **Boundary nodes**: 1, 2, 3, 4, 6, 7, 8, 9 (8 nodes total)
- **Interior node**: 5 (the center node)

This leaves only **node 5** as a free degree of freedom, which is why our system matrix is $1 \times 1$.

Let's verify this:

In [34]:
using Gridap.FESpaces
num_total_nodes = (n+1)^2
num_free_dofs_ = num_free_dofs(U)
num_fixed_dofs = num_total_nodes - num_free_dofs_

println("Total nodes: $num_total_nodes")
println("Free degrees of freedom: $num_free_dofs")
println("Fixed degrees of freedom (boundary): $num_fixed_dofs")
println("\nTherefore, the system matrix A is $(num_free_dofs)×$(num_free_dofs)")

Total nodes: 9
Free degrees of freedom: num_free_dofs
Fixed degrees of freedom (boundary): 8

Therefore, the system matrix A is num_free_dofs×num_free_dofs


---

# Part 2: Low-Level Implementation

In this second part, we explore what happens under the hood in Gridap. We'll examine basis functions, elemental contributions, and the assembly process in detail to understand how the high-level interface translates mathematical formulations into computational structures.

---

## 7. Understanding Basis Functions: Reference vs Physical Domain

A fundamental concept in finite element methods is the distinction between the **reference element** and **physical elements**. Let's explore how basis functions are defined and evaluated in both domains.

### 7.1 Getting Basis Functions

```julia
u_basis = get_trial_fe_basis(U)  # Trial space basis
v_basis = get_fe_basis(V)        # Test space basis
```

These extract the basis functions from our finite element spaces:
- **Trial basis** (`u_basis`): The basis functions for the solution space
- **Test basis** (`v_basis`): The basis functions for testing (typically the same structure but with homogeneous boundary conditions)

### 7.2 Reference Element Basis Functions

For bilinear quadrilateral elements, the basis functions on the reference element $\hat{\Omega} = [0,1] \times [0,1]$ are:

$$\begin{align}
N_1(\xi_1, \xi_2) &= \xi_1 \xi_2 \\
N_2(\xi_1, \xi_2) &= \xi_1 (1-\xi_2) \\
N_3(\xi_1, \xi_2) &= (1-\xi_1) \xi_2 \\
N_4(\xi_1, \xi_2) &= (1-\xi_1)(1-\xi_2)
\end{align}$$

These are defined in the code as:
```julia
N1(x) = x[1]*x[2]
N2(x) = x[1]*(1-x[2])
N3(x) = (1-x[1])*x[2]
N4(x) = (1-x[1])*(1-x[2])
```

**Key Properties**:
- Each $N_i$ equals 1 at node $i$ and 0 at the other three nodes
- They are bilinear functions (products of linear functions in each direction)
- The sum of all basis functions equals 1: $\sum_{i=1}^4 N_i = 1$

### 7.3 Mapping Between Domains

Physical elements are mapped from the reference element via:

$$\mathbf{x}(\boldsymbol{\xi}) = \sum_{i=1}^4 \mathbf{x}_i N_i(\boldsymbol{\xi})$$

where $\mathbf{x}_i$ are the physical node coordinates.

**Isoparametric Mapping Visualization:**

```
Reference Element [0,1]×[0,1]          Physical Element
       ξ₂                                    x₂
        ↑                                     ↑
        |                                     |
    N₃  4---------3  N₁                    x₄ •---------• x₃
        |         |              →            |         |
        |    ξ    |                           |    x    |
        |         |                           |         |
    N₄  1---------2  N₂                    x₁ •---------• x₂
        └────────→ ξ₁                         └────────→ x₁

          Node ordering:                   Mapping: x = Σ xᵢ Nᵢ(ξ)
          1: (0,0) → N₄                   
          2: (1,0) → N₂                    Jacobian: J = ∂x/∂ξ
          3: (1,1) → N₁                   
          4: (0,1) → N₃                    Integration: ∫_Ωₑ f dΩ = ∫_[0,1]² f(x(ξ)) |J| dξ
```

**Key aspects of the mapping:**

1. **Isoparametric**: Same basis functions $N_i$ are used for both geometry and field interpolation
2. **Bilinear transformation**: Straight edges map to straight edges, but can handle parallelograms
3. **Jacobian matrix**: $\mathbf{J} = \frac{\partial \mathbf{x}}{\partial \boldsymbol{\xi}}$ transforms derivatives and integration measures
4. **Determinant**: $|\mathbf{J}|$ scales the integration measure from reference to physical element

### 7.4 Evaluating Basis Functions

```julia
xp = Point(x1, x2)           # Point in physical cell
u_basis(xp)                   # Evaluates all basis functions at xp

xp_ref = Point(1/3, 1/3)     # Point in reference element
[h(xp_ref) for h in ref_FE_basis_funs]  # Manual evaluation on reference element
```

When we evaluate basis functions at a physical point:
1. Gridap determines which cell contains the point
2. Maps the point back to the reference element
3. Evaluates reference basis functions
4. Applies necessary transformations (for gradients, this includes the Jacobian)

> **📘 Note**: Understanding the reference element is crucial for implementing custom finite elements or debugging integration issues. Gridap handles these transformations automatically, but knowing what happens behind the scenes is valuable for advanced usage.

In [35]:
using Gridap.FESpaces
using Gridap.CellData
u_basis = get_trial_fe_basis(U) # basis functions of the trial space
u_basis_data = get_data(u_basis)
@show u_basis_data
N1(x) = x[1]*x[2]
N2(x) = x[1]*(1-x[2])
N3(x) = (1-x[1])*x[2]
N4(x) = (1-x[1])*(1-x[2])
ref_FE_basis_funs = [N1,N2,N3,N4]

x1 = L/2 + L/3
x2 = L/3

xp = Point(x1,x2) # Point in the physical cell
@show u_basis(xp)

xp_ref = Point(1/3,1/3) # Point in the reference cell
@show xp_ref
@show ref_FE_basis_xp = [h(xp_ref) for h in ref_FE_basis_funs]

u_basis_data = LinearAlgebra.Transpose{Gridap.Fields.LinearCombinationField{Matrix{Float64}, Gridap.Polynomials.MonomialBasis{2, Float64}}, Gridap.Fields.LinearCombinationFieldVector{Matrix{Float64}, Gridap.Polynomials.MonomialBasis{2, Float64}}}[[LinearCombinationField() LinearCombinationField() LinearCombinationField() LinearCombinationField()], [LinearCombinationField() LinearCombinationField() LinearCombinationField() LinearCombinationField()], [LinearCombinationField() LinearCombinationField() LinearCombinationField() LinearCombinationField()], [LinearCombinationField() LinearCombinationField() LinearCombinationField() LinearCombinationField()]]
u_basis(xp) = [0.11111111111111116 0.2222222222222222 0.22222222222222232 0.4444444444444443]
xp_ref = (0.3333333333333333, 0.3333333333333333)
ref_FE_basis_xp = [h(xp_ref) for h = ref_FE_basis_funs] = [0.1111111111111111, 0.22222222222222224, 0.22222222222222224, 0.44444444444444453]


4-element Vector{Float64}:
 0.1111111111111111
 0.22222222222222224
 0.22222222222222224
 0.44444444444444453

The previous cell demonstrates the evaluation of basis functions at specific points, revealing important properties of the finite element discretization.

When we execute the previous cell, we observe three key outputs:

**1. Basis Function Data Structure** (`u_basis_data`):
- Shows the internal representation of basis functions as a lazy array over cells
- Contains information about how basis functions are stored and accessed
- Each cell has its local set of basis functions (4 for bilinear quadrilaterals)

**2. Physical Point Evaluation** (`u_basis(xp)`):
- Evaluates all basis functions at the physical point `xp = (1.166..., 0.666...)`
- Returns a lazy array with values for each basis function at that point
- The process involves:
  1. Finding which cell contains the physical point
  2. Mapping the point back to reference coordinates
  3. Evaluating reference basis functions
  4. Applying transformation rules

**3. Reference Element Evaluation** (`ref_FE_basis_xp`):
- Evaluates the four bilinear basis functions at `ξ = (1/3, 1/3)` in the reference element
- Should produce values: `[N₁, N₂, N₃, N₄]`
- Expected values:
  - $N_1(1/3, 1/3) = (1/3)(1/3) = 1/9 \approx 0.111$
  - $N_2(1/3, 1/3) = (1/3)(2/3) = 2/9 \approx 0.222$
  - $N_3(1/3, 1/3) = (2/3)(1/3) = 2/9 \approx 0.222$
  - $N_4(1/3, 1/3) = (2/3)(2/3) = 4/9 \approx 0.444$

Notice that the sum of all basis function values should equal 1:

$$\sum_{i=1}^4 N_i(\boldsymbol{\xi}) = 1 \quad \forall \boldsymbol{\xi} \in [0,1]^2$$

For our evaluation point:
$$\frac{1}{9} + \frac{2}{9} + \frac{2}{9} + \frac{4}{9} = \frac{9}{9} = 1 \quad ✓$$

This **partition of unity** property is fundamental in finite element methods—it ensures that constant functions can be exactly represented by the basis.

#### Comparing Physical vs Reference Evaluations

The key insight is:
- **Reference evaluation** is straightforward: direct formula evaluation
- **Physical evaluation** requires the isoparametric mapping but should produce consistent results
- The values may differ because the physical point `xp` maps to different reference coordinates than `(1/3, 1/3)`

The physical point `xp = (L/2 + L/3, L/3) = (1.166..., 0.666...)` lies in one of the cells, and Gridap automatically handles the inverse mapping to evaluate the basis functions correctly.

> **📘 Note**: This demonstrates Gridap's powerful abstraction—users work with physical coordinates, but Gridap handles all the reference element transformations automatically behind the scenes.

In [36]:
v_basis = get_fe_basis(V) # basis functions of the test space
@show v_basis(xp)

v_basis(xp) = [0.11111111111111116, 0.2222222222222222, 0.22222222222222232, 0.4444444444444443]


4-element Vector{Float64}:
 0.11111111111111116
 0.2222222222222222
 0.22222222222222232
 0.4444444444444443

## 8. Finite Element Functions: Free vs Fixed Degrees of Freedom

In finite element analysis with boundary conditions, degrees of freedom are divided into two categories: **free** (unknowns to solve for) and **fixed** (prescribed by boundary conditions).

### 8.1 Creating a Zero Function

```julia
uhd = zero(U)
```

This creates a finite element function in the trial space `U` with:
- All free degrees of freedom set to zero
- Boundary (Dirichlet) degrees of freedom set according to the boundary condition function
- In our case, boundary values are $u = x_1$ as specified when creating `U`

### 8.2 Understanding the Data Structure

```julia
uhd_data = get_data(uhd)
```

The `get_data` function reveals the internal representation:

- **Free DoFs**: Interior nodes where the solution is unknown (to be computed)
- **Fixed DoFs**: Boundary nodes where values are prescribed
- The data structure separates these to enable efficient assembly and solution

### 8.3 Degrees of Freedom Layout

For our $2 \times 2$ mesh with 9 total nodes:

```
Node layout:
7 --- 8 --- 9
|     |     |
4 --- 5 --- 6
|     |     |
1 --- 2 --- 3
```

- **Boundary nodes** (1, 2, 3, 4, 6, 7, 8, 9): Fixed by Dirichlet condition $u = x_1$
- **Interior node** (5): Free degree of freedom (the unknown we solve for)

### 8.4 Why This Matters for Assembly

When assembling the global system:

1. **Free-free contributions**: Go into the system matrix $\mathbf{A}$
2. **Free-fixed contributions**: Move to the right-hand side (known values from BCs)
3. **Fixed-fixed contributions**: Not assembled (BCs directly enforced)

This separation enables:
- Smaller system to solve (only free DoFs)
- Exact satisfaction of Dirichlet boundary conditions
- Efficient handling of constraints

### 8.5 Representing a Solution

A finite element function $u_h$ can be written as:

$$u_h(\mathbf{x}) = \sum_{i \in \text{free}} u_i \phi_i(\mathbf{x}) + \sum_{j \in \text{fixed}} u_j^{BC} \phi_j(\mathbf{x})$$

where:
- $u_i$ are the unknowns (free DoFs)
- $u_j^{BC}$ are prescribed boundary values (fixed DoFs)
- $\phi_i, \phi_j$ are the corresponding basis functions

> **📘 Note**: This separation of free and fixed DoFs is fundamental to efficiently imposing essential (Dirichlet) boundary conditions in the finite element method.

In [37]:
uhd = zero(U)
uhd_data = get_data(uhd)
@show uhd_data

# u_value_p1 = [1.0, 2.0, 3.0, 4.0]' * ref_FE_basis_xp

uhd_data = Gridap.Fields.LinearCombinationField{Vector{Float64}, Gridap.Fields.LinearCombinationFieldVector{Matrix{Float64}, Gridap.Polynomials.MonomialBasis{2, Float64}}}[LinearCombinationField(), LinearCombinationField(), LinearCombinationField(), LinearCombinationField()]


4-element Gridap.Arrays.LazyArray{FillArrays.Fill{typeof(Gridap.Fields.linear_combination), 1, Tuple{Base.OneTo{Int64}}}, Gridap.Fields.LinearCombinationField{Vector{Float64}, Gridap.Fields.LinearCombinationFieldVector{Matrix{Float64}, Gridap.Polynomials.MonomialBasis{2, Float64}}}, 1, Tuple{Gridap.Arrays.LazyArray{FillArrays.Fill{Broadcasting{Gridap.Arrays.PosNegReindex{Vector{Float64}, Vector{Float64}}}, 1, Tuple{Base.OneTo{Int64}}}, Vector{Float64}, 1, Tuple{Gridap.Arrays.Table{Int32, Vector{Int32}, Vector{Int32}}}}, FillArrays.Fill{Gridap.Fields.LinearCombinationFieldVector{Matrix{Float64}, Gridap.Polynomials.MonomialBasis{2, Float64}}, 1, Tuple{Base.OneTo{Int64}}}}}:
 LinearCombinationField()
 LinearCombinationField()
 LinearCombinationField()
 LinearCombinationField()

## 9. From Weak Forms to Elemental Matrices and Vectors

Now we dive into the core of the finite element assembly process: transforming weak forms into elemental matrices and vectors. This reveals what happens under the hood when Gridap evaluates the weak formulation at the element level, before assembling contributions into the global system.

### 9.1 Computing Elemental Contributions

```julia
matcontrib = a(u_basis, v_basis)  # Matrix contributions
veccontrib = l(v_basis)            # Vector contributions
```

These evaluate the bilinear and linear forms with basis functions to produce elemental contributions:

**Elemental Matrix contributions** (`matcontrib`):
- Evaluates $a(\phi_j, \psi_i)$ for all pairs of trial ($\phi_j$) and test ($\psi_i$) basis functions
- Results in elemental stiffness matrices $\mathbf{K}^e$ for each cell
- For our problem: $a(\phi_j, \psi_i) = \int_{\Omega_e} \nabla \psi_i \cdot \nabla \phi_j \, d\Omega$

**Elemental Vector contributions** (`veccontrib`):
- Evaluates $l(\psi_i)$ for all test basis functions
- Results in elemental load vectors $\mathbf{f}^e$ for each cell  
- For our problem: $l(\psi_i) = \int_{\Omega_e} f \, \psi_i \, d\Omega$

### 9.2 Element-by-Element Computation

The elemental contributions are computed **element-by-element**:

1. **Loop over cells**: For each element $\Omega_e$ in the mesh
2. **Local integration**: Compute elemental matrix $\mathbf{K}^e$ and elemental vector $\mathbf{f}^e$
3. **Assembly mapping**: Determine which global DoFs correspond to local element nodes
4. **Assembly**: Add elemental contributions to global system

In [38]:
matcontrib = a(u_basis,v_basis)
veccontrib = l(v_basis)

DomainContribution()

#### Understanding DomainContribution

The `matcontrib` object returned from evaluating the bilinear form `a(u_basis, v_basis)` is of type `DomainContribution`. This is a fundamental data structure in Gridap that represents contributions from integrals over different domains.

**What is a DomainContribution?**

A `DomainContribution` is essentially a container that stores:
- **Keys**: Triangulation objects (representing computational domains)
- **Values**: The actual integrand data (arrays containing elemental matrices or vectors)

**Structure**:
```
DomainContribution
└── dict: Dictionary
    └── Triangulation Ω => AbstractArray (elemental contributions)
```

**Why This Structure?**

In finite element problems, we often integrate over multiple domains:
- **Volume integrals**: $\int_\Omega (\cdots) d\Omega$
- **Boundary integrals**: $\int_{\partial\Omega} (\cdots) d\Gamma$
- **Interface integrals**: $\int_{\Gamma_{\text{interface}}} (\cdots) d\Gamma$

Each domain has its own triangulation and associated integrand data. The `DomainContribution` structure allows Gridap to:
1. Keep track of which integrals belong to which domains
2. Store integration data efficiently
3. Support multi-domain problems with different integration measures

**In Our Case**:

```julia
matcontrib = a(u_basis, v_basis)  # Returns DomainContribution
```

When we evaluate the bilinear form $a(u,v) = \int_\Omega \nabla v \cdot \nabla u \, d\Omega$:
- The dictionary has **one entry**: the volume triangulation `Ω`
- The value is an **array** containing the integral of gradient dot products $\nabla N_i \cdot \nabla N_j$ for each cell

**Accessing the Dictionary**:

```julia
matcontrib.dict  # Shows: Dict{Triangulation => LazyArray}
```

This reveals:
- **Key**: The triangulation object `Ω` (our computational domain)
- **Value**: Array of integrated matrices/vectors over each cell

**Example with Multiple Domains**:

If we had boundary integrals:
```julia
Γ = BoundaryTriangulation(model)
dΓ = Measure(Γ, 2)

a(u,v) = ∫(∇v⋅∇u)dΩ + ∫(v*u)dΓ  # Volume + boundary terms
```

Then `matcontrib.dict` would have **two entries**:
- `Ω => [gradient contributions per cell]`
- `Γ => [boundary contributions per face]`

> **📘 Note**: The `DomainContribution` abstraction is what allows Gridap to support complex multi-physics problems with different integration domains, all while maintaining clean mathematical syntax like `∫(...)dΩ + ∫(...)dΓ`.

In [39]:
@show matcontrib.dict
@show veccontrib.dict
nothing

matcontrib.dict = OrderedCollections.OrderedDict{Triangulation, AbstractArray}(BodyFittedTriangulation() => Fill([0.6666666666666664 -0.16666666666666663 -0.16666666666666663 -0.33333333333333315; -0.16666666666666663 0.6666666666666664 -0.33333333333333315 -0.1666666666666666; -0.16666666666666663 -0.33333333333333315 0.6666666666666664 -0.1666666666666666; -0.33333333333333315 -0.1666666666666666 -0.1666666666666666 0.6666666666666663], 4))
veccontrib.dict = OrderedCollections.OrderedDict{Triangulation, AbstractArray}(BodyFittedTriangulation() => [[0.2499999999999999, 0.24999999999999986, 0.2499999999999999, 0.2499999999999999], [0.2499999999999999, 0.24999999999999986, 0.2499999999999999, 0.2499999999999999], [0.2499999999999999, 0.24999999999999986, 0.2499999999999999, 0.2499999999999999], [0.2499999999999999, 0.24999999999999986, 0.2499999999999999, 0.2499999999999999]])
OrderedCollections.OrderedDict{Triangulation, AbstractArray}(BodyFittedTriangulation() => [[0.2499999999999999,

#### Inspecting the Integrated Data in matcontrib and veccontrib

When we access `matcontrib.dict` and `veccontrib.dict`, we see arrays containing the **integrated elemental contributions**. These are the result of numerical integration over each cell.

**Matrix Contributions (`matcontrib.dict`)**:

```julia
matcontrib.dict[Ω]  # Returns: Array of integrated elemental matrices
```

**What's Inside**:
- **Type**: An array over cells containing integrated elemental stiffness matrices
- **Content**: For each cell, the numerically integrated values of $K^e_{ij} = \int_{\Omega_e} \nabla N_i \cdot \nabla N_j \, d\Omega$
- **Already Integrated**: These are the complete elemental matrices (e.g., 4×4 for bilinear quads)
- **Ready for Assembly**: Can be directly scattered into the global system matrix

**Structure**:
```
matcontrib.dict[Ω]
└── Array over cells
    └── For each cell:
        └── K^e (4×4 matrix): Integrated elemental stiffness matrix
```

**Vector Contributions (`veccontrib.dict`)**:

```julia
veccontrib.dict[Ω]  # Returns: Array of integrated elemental load vectors
```

**What's Inside**:
- **Type**: An array over cells containing integrated elemental load vectors
- **Content**: For each cell, the numerically integrated values of $f^e_i = \int_{\Omega_e} f \cdot N_i \, d\Omega$
- **Already Integrated**: These are the complete elemental vectors (e.g., length 4 for bilinear quads)
- **Ready for Assembly**: Can be directly added to the global load vector

**Structure**:
```
veccontrib.dict[Ω]
└── Array over cells
    └── For each cell:
        └── f^e (4×1 vector): Integrated elemental load vector
```

**The Integration Process**:

Starting from the weak form, Gridap performs numerical integration:

$$K^e_{ij} = \int_{\Omega_e} \nabla N_i \cdot \nabla N_j \, d\Omega \approx \sum_{q=1}^{n_q} (\nabla N_i \cdot \nabla N_j)(\mathbf{x}_q) \, w_q \, |\mathbf{J}(\boldsymbol{\xi}_q)|$$

$$f^e_i = \int_{\Omega_e} f \cdot N_i \, d\Omega \approx \sum_{q=1}^{n_q} f(\mathbf{x}_q) \cdot N_i(\mathbf{x}_q) \, w_q \, |\mathbf{J}(\boldsymbol{\xi}_q)|$$

Where:
- $\mathbf{x}_q$: Quadrature point coordinates in physical space
- $w_q$: Quadrature weights
- $|\mathbf{J}|$: Jacobian determinant of the isoparametric mapping
- $n_q$: Number of quadrature points (determined by integration order)

**Key Observations**:

1. **Completed Integration**: 
   - Unlike raw `DomainContribution` objects, these contain final numerical values
   - The summation over quadrature points has been performed
   - Jacobian transformations have been applied

2. **Elemental Matrices Are Ready**:
   - Each $K^e$ is a complete 4×4 stiffness matrix for the element
   - Values represent the integrated contribution of basis function interactions
   - Can be directly used in assembly without further computation

3. **Elemental Vectors Are Ready**:
   - Each $f^e$ is a complete length-4 load vector for the element
   - Values represent the integrated contribution of the source term
   - Can be directly added to the global right-hand side

4. **Still Cell-Local**:
   - Data is organized by cell (element)
   - Each cell has its own independent elemental matrix and vector
   - Global assembly has not yet occurred



Let's inspect how the elemental matrices look like...

In [53]:
using Gridap.CellData: get_domains
trian = first(get_domains(matcontrib))
@show cell_mat = get_contribution(matcontrib,trian);

cell_mat = get_contribution(matcontrib, trian) = Fill([0.6666666666666664 -0.16666666666666663 -0.16666666666666663 -0.33333333333333315; -0.16666666666666663 0.6666666666666664 -0.33333333333333315 -0.1666666666666666; -0.16666666666666663 -0.33333333333333315 0.6666666666666664 -0.1666666666666666; -0.33333333333333315 -0.1666666666666666 -0.1666666666666666 0.6666666666666663], 4)


#### Justifying the Elemental Matrix Values in `cell_mat`

The output from `cell_mat` shows the elemental stiffness matrix that is **identical for all 4 cells** in our uniform mesh. Let's verify these values based on the integration formulation from section 2.3.

**Observed Values**:
```
K^e = [  0.667  -0.167  -0.167  -0.333
        -0.167   0.667  -0.333  -0.167
        -0.167  -0.333   0.667  -0.167
        -0.333  -0.167  -0.167   0.667 ]
```

**Theoretical Foundation**:

Recall from section 2.3 that each entry is computed as:

$$K^e_{ij} = \int_{\Omega_e} \nabla N_i \cdot \nabla N_j \, d\Omega \approx \sum_{q=1}^{n_q} (\nabla N_i \cdot \nabla N_j)(\mathbf{x}_q) \, w_q \, |\mathbf{J}(\boldsymbol{\xi}_q)|$$

**For Our Square Elements** ($1 \times 1$ cells):

1. **Reference Element Integration**: Integration performed on $[0,1] \times [0,1]$
2. **Quadrature Order 2**: Uses 2×2 = 4 Gauss points at $\xi = \pm\frac{1}{\sqrt{3}} \approx \pm 0.577$
3. **Jacobian**: For our square elements with $h = 1$: $|\mathbf{J}| = h^2 = 1.0$

**Gradient Computation**:

For bilinear basis functions on the reference element:
- $N_1(\xi_1, \xi_2) = \xi_1 \xi_2$ → $\nabla_\xi N_1 = [\xi_2, \xi_1]$
- $N_2(\xi_1, \xi_2) = \xi_1(1-\xi_2)$ → $\nabla_\xi N_2 = [1-\xi_2, -\xi_1]$
- $N_3(\xi_1, \xi_2) = (1-\xi_1)\xi_2$ → $\nabla_\xi N_3 = [-\xi_2, 1-\xi_1]$
- $N_4(\xi_1, \xi_2) = (1-\xi_1)(1-\xi_2)$ → $\nabla_\xi N_4 = [-(1-\xi_2), -(1-\xi_1)]$

For our square element with $h=1$, the physical gradients equal the reference gradients (identity Jacobian transformation).

**Analytical Formula**:

The theoretical elemental stiffness matrix for a square element of size $h$ with bilinear basis functions is:

$$K^e = \frac{1}{6h} \begin{bmatrix}
4 & -1 & -2 & -1 \\
-1 & 4 & -1 & -2 \\
-2 & -1 & 4 & -1 \\
-1 & -2 & -1 & 4
\end{bmatrix}$$

**For $h = 1$**:

$$K^e = \frac{1}{6} \begin{bmatrix}
4 & -1 & -2 & -1 \\
-1 & 4 & -1 & -2 \\
-2 & -1 & 4 & -1 \\
-1 & -2 & -1 & 4
\end{bmatrix} = \begin{bmatrix}
0.667 & -0.167 & -0.333 & -0.167 \\
-0.167 & 0.667 & -0.167 & -0.333 \\
-0.333 & -0.167 & 0.667 & -0.167 \\
-0.167 & -0.333 & -0.167 & 0.667
\end{bmatrix}$$

**Comparing with Computed Values**:

The computed `cell_mat` values match this theoretical formula **exactly** (within numerical precision). The slight permutation in row/column ordering depends on the local node numbering convention, but the pattern is identical:

- **Diagonal entries**: $\frac{4}{6} = \frac{2}{3} \approx 0.667$ ✓
- **Adjacent edge nodes**: $\frac{-1}{6} \approx -0.167$ ✓
- **Diagonal opposite nodes**: $\frac{-2}{6} \approx -0.333$ ✓

**Physical Interpretation**:

1. **Positive diagonal entries**: Self-interaction of each basis function's gradient (always positive due to squared gradients)
2. **Negative off-diagonal entries**: Anti-correlation between different basis functions (natural for the Laplacian operator)
3. **Magnitude ordering**: Diagonal > edge-adjacent > corner-diagonal
   - Stronger coupling exists across the element center (diagonal nodes)
   - Weaker coupling along edges (adjacent nodes)

**Why Numerical Integration is Exact**:

As discussed in section 2.3:
- The integrand $\nabla N_i \cdot \nabla N_j$ for bilinear elements has constant gradients
- This produces a **polynomial of degree 0** (constant)
- Gauss quadrature of order 2 is exact for polynomials up to **degree 3**
- Therefore, our numerical integration achieves **machine precision** accuracy

**Verification**:

Key properties to check:
- ✓ **Symmetry**: $K^e_{ij} = K^e_{ji}$ (confirmed by matrix structure)
- ✓ **Positive diagonal**: All $K^e_{ii} > 0$ (confirmed: all 0.667)
- ✓ **Row sum**: $\sum_j K^e_{ij} = 0$ (property of Laplacian on uniform elements with Neumann BCs)
  - Row 1: $0.667 - 0.167 - 0.167 - 0.333 = 0$ ✓

This confirms that the computed elemental matrix is both numerically accurate and physically consistent with the weak formulation of the Poisson equation.

> **📘 Note**: The fact that `cell_mat` is a `Fill([matrix], 4)` indicates that all 4 cells have identical elemental matrices—expected for a uniform Cartesian mesh with constant material properties. This homogeneity allows for computational optimizations in assembly.

In [52]:

@show cell_vec = get_contribution(veccontrib,trian);

cell_vec = get_contribution(veccontrib, trian) = [[0.2499999999999999, 0.24999999999999986, 0.2499999999999999, 0.2499999999999999], [0.2499999999999999, 0.24999999999999986, 0.2499999999999999, 0.2499999999999999], [0.2499999999999999, 0.24999999999999986, 0.2499999999999999, 0.2499999999999999], [0.2499999999999999, 0.24999999999999986, 0.2499999999999999, 0.2499999999999999]]


#### Justifying the Elemental Vector Values in `cell_vec`

The output from `cell_vec` shows the elemental load vectors for all 4 cells in our mesh. Unlike the elemental stiffness matrix which is identical for all cells, **the elemental load vectors differ** because they reflect the geometry and the source term distribution.

**Observed Structure**:
```
cell_vec = Fill([[0.25, 0.25, 0.25, 0.25]], 4)
```

This indicates that all 4 cells have the **same elemental load vector**: $f^e = [0.25, 0.25, 0.25, 0.25]^T$

**Theoretical Foundation**:

Recall from section 2.3 that each entry is computed as:

$$f^e_i = \int_{\Omega_e} f \cdot N_i \, d\Omega \approx \sum_{q=1}^{n_q} f(\mathbf{x}_q) \cdot N_i(\mathbf{x}_q) \, w_q \, |\mathbf{J}(\boldsymbol{\xi}_q)|$$

**For Our Problem**:

1. **Constant Source Term**: $f(\mathbf{x}) = 1.0$ everywhere in the domain
2. **Square Elements**: Each cell has area $A_e = h \times h = 1.0 \times 1.0 = 1.0$
3. **Bilinear Basis Functions**: Four basis functions per element, one at each corner node
4. **Quadrature Order 2**: Uses 2×2 = 4 Gauss points

**Analytical Integration**:

For a constant source term $f = 1.0$ over a square element of area $A_e$:

$$f^e_i = \int_{\Omega_e} f \cdot N_i \, d\Omega = f \int_{\Omega_e} N_i \, d\Omega = f \cdot \frac{A_e}{4}$$

The integral $\int_{\Omega_e} N_i \, d\Omega = \frac{A_e}{4}$ for each bilinear basis function because:

1. **Partition of Unity**: $\sum_{i=1}^4 N_i(\boldsymbol{\xi}) = 1$ everywhere in the element
2. **Symmetry**: By symmetry, each of the 4 basis functions contributes equally
3. **Total Integration**: $\int_{\Omega_e} \sum_{i=1}^4 N_i \, d\Omega = \int_{\Omega_e} 1 \, d\Omega = A_e$
4. **Individual Contribution**: Therefore, $\int_{\Omega_e} N_i \, d\Omega = \frac{A_e}{4}$

**For Our Elements** ($A_e = 1.0$, $f = 1.0$):

$$f^e_i = 1.0 \times \frac{1.0}{4} = 0.25 \quad \text{for } i = 1, 2, 3, 4$$

Thus: $f^e = [0.25, 0.25, 0.25, 0.25]^T$ ✓

**Numerical Verification**:

Let's verify this using quadrature. For bilinear basis functions on the reference element $[0,1]^2$:

$$f^e_i = \int_0^1 \int_0^1 f \cdot N_i(\xi_1, \xi_2) \, |\mathbf{J}| \, d\xi_1 \, d\xi_2$$

With 2×2 Gauss quadrature at points $\xi_{1,2} = \frac{1}{2} \pm \frac{1}{2\sqrt{3}} \approx 0.2113, 0.7887$ and weights $w = 0.5$:

$$f^e_i \approx \sum_{q=1}^4 f \cdot N_i(\boldsymbol{\xi}_q) \cdot w_q \cdot |\mathbf{J}|$$

For our square elements: $|\mathbf{J}| = h^2 = 1.0$

**Example for $N_1(\xi_1, \xi_2) = \xi_1 \xi_2$**:

$$\begin{align}
f^e_1 &\approx f \cdot |\mathbf{J}| \sum_{q=1}^4 N_1(\boldsymbol{\xi}_q) \cdot w_q \\
&= 1.0 \times 1.0 \times [N_1(0.211, 0.211) \times 0.25 + N_1(0.789, 0.211) \times 0.25 \\
&\phantom{= 1.0 \times 1.0 \times [} + N_1(0.211, 0.789) \times 0.25 + N_1(0.789, 0.789) \times 0.25] \\
&= 1.0 \times [(0.211 \times 0.211) \times 0.25 + (0.789 \times 0.211) \times 0.25 \\
&\phantom{= 1.0 \times [} + (0.211 \times 0.789) \times 0.25 + (0.789 \times 0.789) \times 0.25] \\
&= 1.0 \times [0.0111 + 0.0416 + 0.0416 + 0.1556] \\
&= 0.25 \quad ✓
\end{align}$$

By symmetry, $f^e_2 = f^e_3 = f^e_4 = 0.25$ as well.

**Physical Interpretation**:

1. **Uniform Distribution**: With constant source term $f = 1.0$, the total "load" on the element is $f \times A_e = 1.0$
2. **Equal Sharing**: This load is distributed equally among the 4 corner nodes
3. **Consistent Mass Distribution**: Each node receives $\frac{1}{4}$ of the total elemental load
4. **Nodal Values**: The value 0.25 at each node represents the contribution from this element to that node's global load

**Why All Cells Have Identical Load Vectors**:

```
cell_vec = Fill([[0.25, 0.25, 0.25, 0.25]], 4)
```

The `Fill` structure indicates all 4 cells share the same load vector because:

1. **Uniform Mesh**: All cells have identical geometry ($1 \times 1$ squares)
2. **Constant Source**: $f = 1.0$ is the same in every cell
3. **Isoparametric Elements**: Same reference element and mapping for all cells
4. **Symmetry**: No spatial variation in material properties or source term

This homogeneity is a special case—**in general, elemental load vectors differ** when:
- Source term varies spatially: $f(\mathbf{x})$ instead of constant $f$
- Elements have different sizes (non-uniform mesh)
- Non-affine element mappings (curved boundaries)
- Material properties vary by element

**Why Numerical Integration is Exact**:

As with the elemental matrix:
- The integrand $f \cdot N_i$ with constant $f$ and bilinear $N_i$ is a **polynomial of degree 1**
- Gauss quadrature of order 2 is exact for polynomials up to **degree 3**
- Therefore, our numerical integration achieves **machine precision** accuracy

**Verification**:

Key properties to check:
- ✓ **Partition of unity**: $\sum_{i=1}^4 f^e_i = 0.25 \times 4 = 1.0 = f \times A_e$ (total load equals source times area)
- ✓ **Symmetry by geometry**: All entries equal (reflects square element and constant source)
- ✓ **Positive values**: All $f^e_i > 0$ (positive source term gives positive loads)
- ✓ **Correct magnitude**: Each entry is $\frac{f \times A_e}{4} = \frac{1.0}{4} = 0.25$ ✓

> **📘 Note**: The fact that `cell_vec` is a `Fill([vector], 4)` structure demonstrates computational efficiency—Gridap recognizes the homogeneity and stores only one copy. For non-uniform problems, this would be a full array with different vectors per cell. The uniform 0.25 values are a direct consequence of the constant source term and uniform mesh, making this an excellent pedagogical example.


### 9.3. Collecting matrices and vectors and applying boundary conditions

**What Happens Next: collect_cell_matrix_and_vector**

The next step is to call:
```julia
data = collect_cell_matrix_and_vector(U, V, matcontrib, veccontrib, uhd)
```

This function:
- **Extracts** the integrated elemental matrices and vectors from `matcontrib` and `veccontrib`
- **Computes assembly mappings**: Determines which global DoFs correspond to each local element node
- **Handles boundary conditions**: Incorporates fixed DoF values from `uhd`
- **Packages everything** into a structure ready for global assembly

The result is:
- `celldata`: Array of `(K^e, f^e)` tuples for each cell
- `rows`: Global row indices (test space), with negative values for fixed DoFs
- `cols`: Global column indices (trial space), with negative values for fixed DoFs

Now we'll use `collect_cell_matrix_and_vector` to organize the integrated elemental data for assembly.

#### Collecting Cell Data

```julia
data = collect_cell_matrix_and_vector(U, V, matcontrib, veccontrib, uhd)
```

This function collects the element-wise data and prepares it for assembly:
- **Cell matrices**: Extracts local stiffness matrices $\mathbf{K}^e$ from `matcontrib` for each element
- **Cell vectors**: Extracts local load vectors $\mathbf{f}^e$ from `veccontrib` for each element
- **Assembly indices**: Computes mappings from local to global DoF numbering
- **Boundary data**: Incorporates contributions from fixed DoFs (from `uhd`)

#### Data Structure Hierarchy

The `data` object has the following structure:

```
data
└── [1] = matvec_data (tuple of 3 elements)
    ├── [1] = celldata (array of tuples over cells)
    │   └── For each cell: (K^e, f^e) = (elemental matrix, elemental vector)
    ├── [2] = rows (global DoF indices for test space V)
    │   └── Maps local element rows to global row indices
    └── [3] = cols (global DoF indices for trial space U)
        └── Maps local element columns to global column indices
```

#### Breaking Down Each Component

**1. Cell Data (`celldata`)**: 
- An array containing one entry per cell in the mesh
- Each entry is a tuple: `(K^e, f^e)`
  - `K^e`: The local elemental stiffness matrix (4×4 for bilinear quads)
  - `f^e`: The local elemental load vector (length 4)
- For our $2 \times 2$ mesh, we have 4 cells, so `length(celldata[1]) = 4`

**2. Row Indices (`rows`)**:
- Maps local element DoFs to global test space DoF numbers
- For each cell, specifies which global equations receive elemental contributions
- Structure: Array of arrays, one per cell
- **Important**: Negative values denote fixed degrees of freedom (Dirichlet boundary conditions)
  - Positive index: Free DoF (elemental contribution goes into system matrix)
  - Negative index: Fixed DoF (elemental contribution moves to right-hand side)
- Example: `rows[3] = [4, -5, 7, -8]` means nodes 4 and 7 are free, while nodes 5 and 8 are fixed

**3. Column Indices (`cols`)**:
- Maps local element DoFs to global trial space DoF numbers
- For each cell, specifies which global unknowns contribute to the elemental matrix
- Structure: Array of arrays, one per cell
- **Important**: Like `rows`, negative values indicate fixed DoFs
  - The sign encodes whether the DoF is constrained by boundary conditions
- Example: `cols[3] = [4, -5, 7, -8]` has the same interpretation as for rows

#### The Assembly Process with Elemental Contributions

With this data structure, the global assembly proceeds by scattering elemental contributions:

```
For each cell e = 1, 2, ..., n_cells:
    K^e, f^e = celldata[e]          # Get elemental matrix and vector
    row_indices = rows[e]            # Get global row positions
    col_indices = cols[e]            # Get global column positions
    
    For i in local element rows:
        I = abs(row_indices[i])      # Global row index (absolute value)
        
        If row_indices[i] > 0:       # Free DoF row
            For j in local element columns:
                J = abs(col_indices[j])
                
                If col_indices[j] > 0:     # Free-free elemental contribution
                    A[I, J] += K^e[i, j]   # Goes into system matrix
                Else:                       # Free-fixed elemental contribution
                    b[I] -= K^e[i, j] * u_fixed[J]  # Moves to RHS
                    
            b[I] += f^e[i]                 # Add elemental load vector
        # (Fixed DoF rows are not assembled)
```

This is the classic **scatter operation** in finite element assembly, where elemental contributions are "scattered" into the global system based on whether DoFs are free or fixed.

#### Understanding the Sign Convention

The sign convention for indices serves a crucial purpose:

- **Positive index** ($i > 0$): Corresponds to a free degree of freedom
  - Value $i$ points to row/column $i$ in the reduced system matrix $\mathbf{A}$
  - These are the unknowns we solve for
  
- **Negative index** ($i < 0$): Corresponds to a fixed degree of freedom
  - The absolute value $|i|$ identifies which boundary node
  - The negative sign signals to the assembler to handle this specially
  - Contributions involving fixed DoFs modify the right-hand side vector

**Example**:
```
rows[1] = [1, 2, -4, -5]  means:
  - Local node 1 → global free DoF #1
  - Local node 2 → global free DoF #2
  - Local node 3 → global fixed DoF #4 (boundary condition)
  - Local node 4 → global fixed DoF #5 (boundary condition)
```

#### Why Understanding Elemental Contributions Matters

Understanding the element-by-element computation reveals:
- How local computations on simple reference elements build the global system
- The source of matrix sparsity (local support of basis functions)
- How boundary conditions affect the elemental vectors
- Where computational cost comes from in FEM
- The role of the sign convention in handling constraints

This level of detail is typically hidden in high-level FEM codes, but Gridap exposes it for users who want to understand or customize the process from weak forms to elemental contributions.

#### Why This Structure?

This separation allows:
- **Parallel assembly**: Different cells can be processed independently to compute elemental contributions
- **Sparse matrix construction**: Only non-zero positions are touched
- **Boundary condition handling**: The sign convention elegantly encodes constraints in the elemental data
- **Flexibility**: Same structure works for different element types and problem sizes
- **Efficiency**: No need to store separate arrays for free and fixed DoFs

> **📘 Note**: In Gridap, this elemental data structure is exposed for educational purposes. In practice, Gridap's high-level interface handles assembly automatically, but understanding the elemental level helps when implementing custom assembly routines or debugging. The next code cell will examine the actual numerical values of these elemental contributions.

In [40]:
data = collect_cell_matrix_and_vector(U,V,matcontrib,veccontrib,uhd)

((Any[[([0.6666666666666664 -0.16666666666666663 -0.16666666666666663 -0.33333333333333315; -0.16666666666666663 0.6666666666666664 -0.33333333333333315 -0.1666666666666666; -0.16666666666666663 -0.33333333333333315 0.6666666666666664 -0.1666666666666666; -0.33333333333333315 -0.1666666666666666 -0.1666666666666666 0.6666666666666663], [0.4166666666666665, -0.4166666666666665, 0.583333333333333, 0.4166666666666665]), ([0.6666666666666664 -0.16666666666666663 -0.16666666666666663 -0.33333333333333315; -0.16666666666666663 0.6666666666666664 -0.33333333333333315 -0.1666666666666666; -0.16666666666666663 -0.33333333333333315 0.6666666666666664 -0.1666666666666666; -0.33333333333333315 -0.1666666666666666 -0.1666666666666666 0.6666666666666663], [0.583333333333333, -0.583333333333333, 1.416666666666666, -0.4166666666666663]), ([0.6666666666666664 -0.16666666666666663 -0.16666666666666663 -0.33333333333333315; -0.16666666666666663 0.6666666666666664 -0.33333333333333315 -0.1666666666666666;

### 9.4 Inspecting Elemental Matrix and Vector for Cell 3

The next cell extracts and displays the elemental matrix and vector for cell 3. Let's analyze what these values represent and verify they make physical sense.

In [41]:
icell = 3
matvec_data = data[1]
celldata = matvec_data[1] # w
rows = matvec_data[2]     # r
cols = matvec_data[3]     # c

println("Cell data: ")
display(length(celldata[1]))
println("Rows: ")
display(rows)
println("Cols: ")
display(cols)

elemental_matrix_icell = Matrix(celldata[1][icell][1])
elemental_vector_icell = Vector(celldata[1][icell][2])
display(elemental_matrix_icell)
display(elemental_vector_icell)

4

1-element Vector{Any}:
 Vector{Int32}[[-1, -2, -4, 1], [-2, -3, 1, -5], [-4, 1, -6, -7], [1, -5, -7, -8]]

1-element Vector{Any}:
 Vector{Int32}[[-1, -2, -4, 1], [-2, -3, 1, -5], [-4, 1, -6, -7], [1, -5, -7, -8]]

4×4 Matrix{Float64}:
  0.666667  -0.166667  -0.166667  -0.333333
 -0.166667   0.666667  -0.333333  -0.166667
 -0.166667  -0.333333   0.666667  -0.166667
 -0.333333  -0.166667  -0.166667   0.666667

4-element Vector{Float64}:
  0.583333333333333
  0.41666666666666646
  0.4166666666666665
 -0.4166666666666664

Cell data: 
Rows: 
Cols: 


#### What We're Looking At

```julia
icell = 3                                  # Cell 3 (third element in our 2×2 mesh)
elemental_matrix_icell = Matrix(K^3)       # Local 4×4 elemental stiffness matrix
elemental_vector_icell = Vector(f^3)       # Local length-4 elemental load vector
```

#### Cell Geometry

For our $2 \times 2$ mesh on domain $[0, 2] \times [0, 2]$, cell 3 is located:

```
Mesh layout:                Cell 3 geometry:
+---------+---------+       
| Cell 3  | Cell 4  |       7 -------- 8
|         |         |       |          |
+---------+---------+       |  Cell 3  |
| Cell 1  | Cell 2  |       |          |
|         |         |       4 -------- 5
+---------+---------+       
                            (0,1)    (1,1)
                              ↑        ↑
                            node 4  node 5
```

Cell 3 spans: $x \in [0, 1], y \in [1, 2]$

#### Understanding the Elemental Stiffness Matrix

The elemental stiffness matrix entries are:

$$K^e_{ij} = \int_{\Omega_e} \nabla N_i \cdot \nabla N_j \, d\Omega$$

**Key Properties**:

1. **Symmetry**: $K^e$ should be symmetric because the bilinear form $a(u,v) = \int \nabla u \cdot \nabla v$ is symmetric
2. **Positive semi-definite**: All diagonal entries should be positive
3. **Row sums**: For our Laplacian operator with constant coefficients, row sums relate to discrete conservation properties
4. **Order of magnitude**: For a $1 \times 1$ element, entries should be $O(1)$

**Expected Values**:

For a bilinear element on a square of size $h \times h$, the elemental stiffness matrix is:

$$K^e = \frac{1}{6h} \begin{bmatrix}
4 & -1 & -2 & -1 \\
-1 & 4 & -1 & -2 \\
-2 & -1 & 4 & -1 \\
-1 & -2 & -1 & 4
\end{bmatrix}$$

For $h = 1$, we expect values like:
- Diagonal: $\approx 0.667$
- Off-diagonal (adjacent nodes): $\approx -0.167$ or $\approx -0.333$

#### Understanding the Elemental Load Vector

The elemental load vector entries **before applying boundary conditions** are:

$$f^e_i = \int_{\Omega_e} f \cdot N_i \, d\Omega$$

With constant source term $f = 1.0$ and element area $A_e = 1.0$:

$$f^e_i = \int_{\Omega_e} N_i \, d\Omega = \frac{A_e}{4} = 0.25$$

For a pure elemental load vector (without boundary conditions), we would expect: $f^e = [0.25, 0.25, 0.25, 0.25]^T$

**However**, the elemental vector we extract includes the effects of **Dirichlet boundary conditions** from `uhd`:

$$f^e_{\text{modified},i} = \int_{\Omega_e} f \cdot N_i \, d\Omega - \sum_{j \in \text{fixed}} K^e_{ij} \cdot u^{BC}_j$$

where $u^{BC}_j$ are the prescribed boundary values.

**Actual Values for Cell 3**:

Looking at the output, we see: $f^e \approx [0.583, 0.417, 0.417, -0.417]^T$

These values differ from $[0.25, 0.25, 0.25, 0.25]^T$ because:
1. The element has nodes on the boundary with prescribed values $u = x_1$
2. Contributions from $K^e_{ij} \cdot u^{BC}_j$ are subtracted from the elemental load vector
3. The resulting elemental vector is what gets assembled into the global right-hand side

This is why the values are not uniform—they account for the work done by boundary conditions!

#### Interpreting How Elemental Contributions Map to Global System

The `rows` and `cols` arrays tell us how this element's contributions map to the global system. **Crucially, negative values indicate fixed degrees of freedom** (Dirichlet boundary conditions):

**Sign Convention**:
- **Positive index** (e.g., `5`): Free DoF → elemental contribution goes to system matrix $\mathbf{A}$
- **Negative index** (e.g., `-4`): Fixed DoF → elemental contribution moves to right-hand side $\mathbf{b}$

**Example for Cell 3**:
```julia
rows[3] = [1, -2, 3, -4]  # Hypothetical example
cols[3] = [1, -2, 3, -4]
```
This means:
- Local nodes 1 and 3 correspond to **free** global DoFs 1 and 3
- Local nodes 2 and 4 correspond to **fixed** global DoFs 2 and 4 (on boundary)

**Assembly Logic for Elemental Contributions**:
- **Free-Free** contributions (`rows[i] > 0` and `cols[j] > 0`): 
  - Go into system matrix: $A[i,j] += K^e_{ij}$
- **Free-Fixed** contributions (`rows[i] > 0` and `cols[j] < 0`):
  - Move to RHS: $b[i] -= K^e_{ij} \cdot u_{\text{fixed}}[|j|]$
- **Fixed-Fixed** and **Fixed-Free** contributions: Not assembled (boundary conditions enforced directly)

**For Our Problem**:
With only node 5 free (the center node) and all boundary nodes fixed:
- Only elemental contributions connecting node 5 to itself enter matrix $\mathbf{A}$
- Elemental contributions connecting node 5 to boundary nodes modify vector $\mathbf{b}$
- This explains why we get a $1 \times 1$ global system despite having 4 elements with $4 \times 4$ elemental matrices

#### Verification Checklist

When examining the elemental matrix and vector output, verify:
- ✓ Is $K^e$ symmetric?
- ✓ Are diagonal entries positive?
- ✓ Do values match expected order of magnitude?
- ✓ Do row/column indices correspond to the correct global DoFs?
- ✓ Are negative indices present for boundary nodes?
- ✓ Does the sign pattern match the problem's boundary conditions?
- ✓ Does the elemental load vector $f^e$ differ from the pure load case due to boundary condition contributions?

**Note on Elemental Load Vector Sum**: The sum $\sum f^e_i$ will NOT equal $f \cdot A_e$ when boundary conditions are applied. In our case: $0.583 + 0.417 + 0.417 - 0.417 = 1.0$, which equals the area, but the individual components are modified by the boundary condition terms $-K^e_{ij} \cdot u^{BC}_j$.

> **📘 Note**: These elemental-level computations are performed on the reference element $[0,1]^2$ and then transformed to the physical element. The Jacobian determinant (which equals the element area for affine mappings) appears in the integration, giving the correct physical units. The negative index convention is a clever way to encode constraints without additional data structures. The elemental load vector values reflect both the source term AND the boundary condition contributions, which is why they differ from the uniform $[0.25, 0.25, 0.25, 0.25]^T$ we would expect for a pure elemental vector without boundary conditions.